# 1. Cleaning and Processing Consumer Complaints Dataset

## Introduction

This dataset represents consumer complaints about financial products and services collected by the Consumer Financial Protection Bureau (CFPB). The dataset includes information on the type of product, issue, company response, and more.

The purpose of this part of the project is to clean and process the first of three datasets. This dataset requires:

1. **Data Inspection**: Understanding its structure and identifying key fields.
2. **Data Cleaning**: Removing duplicates, handling missing values, and ensuring consistent formatting.
3. **Feature Preparation**: Preparing fields for further analysis and integration with other files.

This work is the foundation for integrating and analyzing multiple datasets, enabling meaningful insights into consumer financial complaints.


In [2]:
import numpy as np
import pandas as pd
import json
import os

In [3]:
# Define the file path
file_path = r'C:\Users\Zana\Desktop\portfolio_projects\project_4\complaints.json'

In [4]:
# Load the JSON data into a pandas DataFrame
with open(file_path, 'r') as f:
    data = json.load(f)

In [5]:
# If the JSON structure is flat (list of dictionaries), you can directly create a DataFrame
df = pd.json_normalize(data)

In [6]:
df.head()

,took,timed_out,_shards.total,_shards.successful,_shards.skipped,_shards.failed,hits.total.value,hits.total.relation,hits.max_score,hits.hits,...,_meta.total_record_count,_meta.is_data_stale,_meta.has_data_issue,_meta.break_points.2,_meta.break_points.3,_meta.break_points.4,_meta.break_points.5,_meta.break_points.6,_meta.break_points.7,_meta.break_points.8
0,1373,False,5,5,0,0,3093136,eq,None,"[{'_index': 'complaint-public-v1', '_type': '_...",...,3093136,False,False,"[1.0, 999878]","[1.0, 99976]","[1.0, 999652]","[1.0, 999589]","[1.0, 999492]","[1.0, 999398]","[1.0, 99928]"


# Inspected Raw JSON Structure

In [7]:
# Load and inspect the raw JSON data
import json

file_path = r'C:\Users\Zana\Desktop\portfolio_projects\project_4\complaints.json'

# Open the file and print the first few entries
with open(file_path, 'r') as f:
    data = json.load(f)

# Print the first few entries or keys
print(json.dumps(data, indent=4))  # Pretty print the JSON structure

{
    "took": 1373,
    "timed_out": false,
    "_shards": {
        "total": 5,
        "successful": 5,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 3093136,
            "relation": "eq"
        },
        "max_score": null,
        "hits": [
            {
                "_index": "complaint-public-v1",
                "_type": "_doc",
                "_id": "99999",
                "_score": 1.0,
                "_source": {
                    "product": "Bank account or service",
                    "complaint_what_happened": "",
                    "date_sent_to_company": "2013-11-18T12:00:00-05:00",
                    "issue": "Deposits and withdrawals",
                    "sub_product": "Checking account",
                    "zip_code": "94605",
                    "tags": "Older American",
                    "has_narrative": false,
                    "complaint_id": "99999",
                    "timely": "Yes",
  

In [8]:
# Print the first 100 characters or specific keys from the JSON file
with open(file_path, 'r') as f:
    data = json.load(f)

# Print only a part of the data to inspect the structure
print(json.dumps(data)[:500])  # Adjust the slice as needed

# Alternatively, print just the keys
if isinstance(data, dict):
    print(data.keys())
elif isinstance(data, list):
    print(data[0].keys())  # Assuming it's a list of dictionaries

{"took": 1373, "timed_out": false, "_shards": {"total": 5, "successful": 5, "skipped": 0, "failed": 0}, "hits": {"total": {"value": 3093136, "relation": "eq"}, "max_score": null, "hits": [{"_index": "complaint-public-v1", "_type": "_doc", "_id": "99999", "_score": 1.0, "_source": {"product": "Bank account or service", "complaint_what_happened": "", "date_sent_to_company": "2013-11-18T12:00:00-05:00", "issue": "Deposits and withdrawals", "sub_product": "Checking account", "zip_code": "94605", "ta
dict_keys(['took', 'timed_out', '_shards', 'hits', 'aggregations', '_meta'])


In [9]:
# Extract the list of complaint records from 'hits'
complaint_records = data['hits']['hits']

In [10]:
# Flatten the nested JSON into a DataFrame (extracting fields from '_source')
df = pd.json_normalize(complaint_records, sep='_', record_path=None, 
                       meta=['_id', '_score', '_index'], 
                       meta_prefix='meta_', errors='ignore')

In [11]:
# Focus only on the fields inside '_source'
df = df[df.columns[df.columns.str.startswith('_source')]]

In [12]:
# Flatten the '_source' field itself
df.columns = df.columns.str.replace('_source.', '')

In [13]:
df.head()

,_source_product,_source_complaint_what_happened,_source_date_sent_to_company,_source_issue,_source_sub_product,_source_zip_code,_source_tags,_source_has_narrative,_source_complaint_id,_source_timely,_source_consumer_consent_provided,_source_company_response,_source_submitted_via,_source_company,_source_date_received,_source_state,_source_consumer_disputed,_source_company_public_response,_source_sub_issue
0,Bank account or service,,2013-11-18T12:00:00-05:00,Deposits and withdrawals,Checking account,94605,Older American,False,99999,Yes,N/A,Closed with explanation,Phone,WELLS FARGO & COMPANY,2012-06-12T12:00:00-05:00,CA,No,None,None
1,Debt collection,,2014-09-04T12:00:00-05:00,Communication tactics,I do not know,93305,None,False,999988,Yes,N/A,Closed,Web,"Kimball, Tirey & St. John LLP",2014-08-25T12:00:00-05:00,CA,No,None,Used obscene/profane/abusive language
2,Mortgage,,2014-08-28T12:00:00-05:00,"Application, originator, mortgage broker",Conventional fixed mortgage,32818,None,False,999986,Yes,N/A,Closed with explanation,Phone,JPMORGAN CHASE & CO.,2014-08-25T12:00:00-05:00,FL,Yes,None,None
3,Credit card,,2012-06-15T12:00:00-05:00,Advertising and marketing,None,98221,Older American,False,99998,Yes,N/A,Closed with explanation,Web,CAPITAL ONE FINANCIAL CORPORATION,2012-06-12T12:00:00-05:00,WA,No,None,None
4,Bank account or service,,2012-06-21T12:00:00-05:00,Problems caused by my funds being low,Checking account,19140,"Older American, Servicemember",False,99997,Yes,N/A,Closed,Postal mail,"SANTANDER BANK, NATIONAL ASSOCIATION",2012-06-12T12:00:00-05:00,PA,No,None,None


In [16]:
df.shape

(25, 19)

In [17]:
df.columns

Index(['_source_product', '_source_complaint_what_happened',
       '_source_date_sent_to_company', '_source_issue', '_source_sub_product',
       '_source_zip_code', '_source_tags', '_source_has_narrative',
       '_source_complaint_id', '_source_timely',
       '_source_consumer_consent_provided', '_source_company_response',
       '_source_submitted_via', '_source_company', '_source_date_received',
       '_source_state', '_source_consumer_disputed',
       '_source_company_public_response', '_source_sub_issue'],
      dtype='object')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   _source_product                    25 non-null     object
 1   _source_complaint_what_happened    25 non-null     object
 2   _source_date_sent_to_company       25 non-null     object
 3   _source_issue                      25 non-null     object
 4   _source_sub_product                16 non-null     object
 5   _source_zip_code                   25 non-null     object
 6   _source_tags                       7 non-null      object
 7   _source_has_narrative              25 non-null     bool  
 8   _source_complaint_id               25 non-null     object
 9   _source_timely                     25 non-null     object
 10  _source_consumer_consent_provided  25 non-null     object
 11  _source_company_response           25 non-null     object
 12  _source_su

In [19]:
df.isnull().sum()

_source_product                       0
_source_complaint_what_happened       0
_source_date_sent_to_company          0
_source_issue                         0
_source_sub_product                   9
_source_zip_code                      0
_source_tags                         18
_source_has_narrative                 0
_source_complaint_id                  0
_source_timely                        0
_source_consumer_consent_provided     0
_source_company_response              0
_source_submitted_via                 0
_source_company                       0
_source_date_received                 0
_source_state                         0
_source_consumer_disputed             0
_source_company_public_response      25
_source_sub_issue                    18
dtype: int64

In [20]:
missing_percentage = df.isnull().sum() / len(df) * 100
print(missing_percentage)

_source_product                        0.0
_source_complaint_what_happened        0.0
_source_date_sent_to_company           0.0
_source_issue                          0.0
_source_sub_product                   36.0
_source_zip_code                       0.0
_source_tags                          72.0
_source_has_narrative                  0.0
_source_complaint_id                   0.0
_source_timely                         0.0
_source_consumer_consent_provided      0.0
_source_company_response               0.0
_source_submitted_via                  0.0
_source_company                        0.0
_source_date_received                  0.0
_source_state                          0.0
_source_consumer_disputed              0.0
_source_company_public_response      100.0
_source_sub_issue                     72.0
dtype: float64


In [21]:
# Option 1: Drop rows with missing values in specific columns
df = df.dropna(subset=['_source_date_sent_to_company'])

# Option 2: Fill missing values in categorical fields with 'Unknown'
df['_source_sub_product'] = df['_source_sub_product'].fillna('Unknown')

# Option 3: Drop columns with too many missing values (if necessary)
df = df.drop(columns=['_source_sub_issue'], errors='ignore')

In [22]:
# Check for duplicates
duplicates = df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

# Drop duplicate rows if necessary
df = df.drop_duplicates()

Number of duplicate rows: 0


In [23]:
# Convert the 'date_sent_to_company' column to datetime
df['_source_date_sent_to_company'] = pd.to_datetime(df['_source_date_sent_to_company'], errors='coerce')

# Check for any remaining rows with invalid dates
df = df.dropna(subset=['_source_date_sent_to_company'])

# Clean categorical columns (e.g., product categories)
df['_source_product'] = df['_source_product'].str.strip().str.lower()

In [24]:
# Example: Detect outliers in numeric fields (if applicable)
df['_source_complaint_id'] = pd.to_numeric(df['_source_complaint_id'], errors='coerce')

# You can plot or inspect summary statistics to detect outliers
df['_source_complaint_id'].describe()

count        25.000000
mean     675955.000000
std      440877.791061
min       99989.000000
25%       99997.000000
50%      999902.000000
75%      999944.000000
max      999988.000000
Name: _source_complaint_id, dtype: float64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype                    
---  ------                             --------------  -----                    
 0   _source_product                    25 non-null     object                   
 1   _source_complaint_what_happened    25 non-null     object                   
 2   _source_date_sent_to_company       25 non-null     datetime64[ns, UTC-05:00]
 3   _source_issue                      25 non-null     object                   
 4   _source_sub_product                25 non-null     object                   
 5   _source_zip_code                   25 non-null     object                   
 6   _source_tags                       7 non-null      object                   
 7   _source_has_narrative              25 non-null     bool                     
 8   _source_complaint_id               25 non-null     int64                

In [26]:
df.head()

,_source_product,_source_complaint_what_happened,_source_date_sent_to_company,_source_issue,_source_sub_product,_source_zip_code,_source_tags,_source_has_narrative,_source_complaint_id,_source_timely,_source_consumer_consent_provided,_source_company_response,_source_submitted_via,_source_company,_source_date_received,_source_state,_source_consumer_disputed,_source_company_public_response
0,bank account or service,,2013-11-18 12:00:00-05:00,Deposits and withdrawals,Checking account,94605,Older American,False,99999,Yes,N/A,Closed with explanation,Phone,WELLS FARGO & COMPANY,2012-06-12T12:00:00-05:00,CA,No,None
1,debt collection,,2014-09-04 12:00:00-05:00,Communication tactics,I do not know,93305,None,False,999988,Yes,N/A,Closed,Web,"Kimball, Tirey & St. John LLP",2014-08-25T12:00:00-05:00,CA,No,None
2,mortgage,,2014-08-28 12:00:00-05:00,"Application, originator, mortgage broker",Conventional fixed mortgage,32818,None,False,999986,Yes,N/A,Closed with explanation,Phone,JPMORGAN CHASE & CO.,2014-08-25T12:00:00-05:00,FL,Yes,None
3,credit card,,2012-06-15 12:00:00-05:00,Advertising and marketing,Unknown,98221,Older American,False,99998,Yes,N/A,Closed with explanation,Web,CAPITAL ONE FINANCIAL CORPORATION,2012-06-12T12:00:00-05:00,WA,No,None
4,bank account or service,,2012-06-21 12:00:00-05:00,Problems caused by my funds being low,Checking account,19140,"Older American, Servicemember",False,99997,Yes,N/A,Closed,Postal mail,"SANTANDER BANK, NATIONAL ASSOCIATION",2012-06-12T12:00:00-05:00,PA,No,None


In [27]:
# Save the cleaned DataFrame to a JSON file
output_file_path = r'C:\Users\Zana\Desktop\portfolio_projects\project_4\cleaned_complaints.json'
df.to_json(output_file_path, orient='records', date_format='iso')
print(f"Cleaned data has been saved to {output_file_path}")

Cleaned data has been saved to C:\Users\Zana\Desktop\portfolio_projects\project_4\cleaned_complaints.json


## Sample Output of Cleaned Dataset

Below is a preview of the cleaned dataset after processing:

| Product                    | Issue                          | Company Response       | State | Complaint ID | Timely Response | Submitted Via | Tags            |
|----------------------------|-------------------------------|------------------------|-------|--------------|-----------------|---------------|-----------------|
| Credit card                | Billing disputes              | Closed with explanation| CA    | 99999        | Yes             | Web           | Older American  |
| Mortgage                   | Application, originator issue | Closed with relief     | FL    | 999988       | Yes             | Phone         | None            |
| Debt collection            | Communication tactics         | Closed                | NY    | 999986       | No              | Postal mail   | Servicemember   |
| Checking account           | Deposits and withdrawals      | Closed with explanation| TX    | 99998        | Yes             | Web           | None            |

### Data Quality Improvements:
- **Missing Values**: Handled by filling with default or `Unknown`.
- **Duplicates**: Removed 0 duplicates.
- **Standardized Fields**: Cleaned categorical fields (e.g., state names, products).

This cleaned dataset is now ready for analysis and further integration.
